# Estimating Stop Frequency

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('test')

# Load data and prep model for estimation

In [3]:
modelname = "stop_frequency"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/activitysim/activitysim/estimation/larch/general.py:321: UserWarning: coefficient dataframe missing 'constrain' column, setting all to 'F'
  warnings.warn("coefficient dataframe missing 'constrain' column, setting all to 'F'")


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

In [4]:
spec_segments = [i.primary_purpose for i in data.settings.SPEC_SEGMENTS]
spec_segments

['work',
 'school',
 'univ',
 'social',
 'shopping',
 'eatout',
 'escort',
 'othmaint',
 'othdiscr',
 'atwork']

## Coefficients

There is one meta-coefficients dataframe for this component, which contains
parameters for all the matching coefficients in the various segmented 
files. When different segments have the same named coefficient with the same
value, it is assumed they should be estimated jointly.  If they have the same name
but different values in the coefficient files, then they are re-estimated
independently.

In [5]:
data.coefficients

,value,constrain
coefficient_name,,
coef_middle_to_low_income_hh,0.170,F
coef_mid_to_high_income_hh,0.230,F
coef_high_income_hh,0.240,F
coef_number_of_hh_persons,-0.310,F
coef_number_of_students_in_hh,0.210,F
...,...,...
coef_alternative_specific_constant_for_return_stops_0out_3in_atwork,-6.210,F
coef_alternative_specific_constant_for_outbound_stops_1out_0in_atwork,-3.896,F
coef_alternative_specific_constant_for_the_total_number_of_stops_1out_3in_atwork,2.127,F


## Utility specification

The utility spec files are unique to each segment model.  The estimation mode larch pre-processor
for the stop frequency model modifies the spec files to account for jointly re-estimated
parameters.

In [6]:
data.spec[0]

,Label,Description,Expression,0out_0in,0out_1in,0out_2in,0out_3in,1out_0in,1out_1in,1out_2in,1out_3in,2out_0in,2out_1in,2out_2in,2out_3in,3out_0in,3out_1in,3out_2in,3out_3in
0,util_middle_to_low_income_hh,Middle to Low Income HH,(income_in_thousands>19999) & (income_in_thous...,NaN,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh,coef_middle_to_low_income_hh
1,util_mid_to_high_income_hh,Mid to High Income HH,(income_in_thousands>=50000) & (income_in_thou...,NaN,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh,coef_mid_to_high_income_hh
2,util_high_income_hh,High Income HH,(income_in_thousands>=100000),NaN,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh,coef_high_income_hh
3,util_number_of_hh_persons,Number of HH Persons,hhsize,NaN,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons,coef_number_of_hh_persons
4,util_number_of_full_time_workers_in_hh,Number of full time workers in HH,num_full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,util_number_of_students_in_hh,Number of Students in HH,num_student,NaN,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh,coef_number_of_students_in_hh
6,util_num_kids_between_0_and_4_including_years_old,Num Kids between 0 and 4 (including) years old,num_age_0_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,util_presence_of_kids_between_0_and_4_includin...,Presence of Kids between 0 and 4 (including) y...,(num_age_0_4 > 0),NaN,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...,coef_presence_of_kids_between_0_and_4_includin...
8,util_num_kids_between_5_and_15_including_years...,Num kids between 5 and 15 (including) years old,num_age_5_15,NaN,coef_num_kids_between_5_and_15_including_years...,coef_num_kids_between_5_and_15_including_years...,coef_num_kids_

## Chooser data

The chooser data is unique to each segment model. 

In [7]:
data.chooser_data[0]

,model_choice,override_choice,util_middle_to_low_income_hh,util_mid_to_high_income_hh,util_high_income_hh,util_number_of_hh_persons,util_number_of_full_time_workers_in_hh,util_number_of_students_in_hh,util_num_kids_between_0_and_4_including_years_old,util_presence_of_kids_between_0_and_4_including_years_old,...,tour_mode_is_drive_transit,tour_mode_is_non_motorized,num_school_tours,num_univ_tours,num_atwork_subtours,num_hh_shop_tours,num_hh_maint_tours,hhacc,pracc,destination_area_type
tour_id,,,,,,,,,,,,,,,,,,,,,
2961920,0out_0in,0out_0in,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,0,0,0,0,0,6.859067,7.751024,0
2970120,2out_1in,2out_1in,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,0,0,0,0,0,0.000000,0.000000,1
2998943,0out_0in,0out_0in,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,0,0,1,0,0,6.024261,7.519040,2
3013252,0out_0in,0out_0in,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,0,0,0,0,0,5.672122,6.595380,1
3015794,0out_0in,0out_0in,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,False,False,0,0,0,0,0,6.252582,5.865708,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308070309,0out_0in,0out_0in,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,False,True,0,0,0,0,0,8.181251,8.326834,0
308073875,1out_0in,1out_0in,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,0,0,0,0,0,0.000000,0.000000,2
308090603,0out_0in,0out_0in,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,False,False,0,0,0,0,0,7.629996,7.282405,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [8]:
model.estimate(method='SLSQP', options={"maxiter": 1000})

req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
-2.139,-2.139000,-2.139,-2.139,-2.139,-2.139,1,,-2.139000
-3.934,-3.934000,-3.934,-3.934,-3.934,-3.934,1,,-3.934000
coef_alternative_specific_constant_for_outbound_stops_1out_0in,-0.741225,-0.833,0.000,NaN,NaN,0,,-0.741225
coef_alternative_specific_constant_for_outbound_stops_2out_0in,-2.594489,-2.613,0.000,NaN,NaN,0,,-2.594489
coef_alternative_specific_constant_for_return_stops_0out_1in,-0.447476,-0.445,0.000,NaN,NaN,0,,-0.447476
...,...,...,...,...,...,...,...,...
coef_primary_destination_accessibility_log_of_it_,0.170704,0.180,0.000,NaN,NaN,0,,0.170704
coef_subtour_departure_less_than_or_equal_to_11am,0.266339,0.310,0.000,NaN,NaN,0,,0.266339
coef_subtour_distance_in_miles_from_tour_destination_to_subtour_primary_destination_one_way_,-0.152384,0.020,0.000,NaN,NaN,0,,-0.152384
coef_subtour_duration_in_hours_integer_,0.681811,0.560,0.000,NaN,NaN,0,,0.681811


<ipython-input-8-c0c5a8a8ae55>:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  model.estimate(method='SLSQP', options={"maxiter": 1000})
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.0254868525048906e-31 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: -2.139                                                                                         -2.139000
┃                   -3.934                                                                                         -3.934000
┃                   coef_alternative_specific_constant_for_outbound_stops_1out_0in                                 -0.741225
┃                   coef_alternative_specific_constant_for_outbound_stops_2out_0in                                 -2.594489
┃                   coef_alternative_specific_constant_for_return_stops_0out_1in                                   -0.447476
┃                                                                                                                     ...   
┃                   coef_primary_destination_accessibility_log_of_it_                                               0.170704
┃                   coef_subtour_departure_less_than_or_equal_to_11am                                               0.266339
┃                   coef_subtour_distance_in_miles_from_tour_destination_to_subtour_primary_destination_one_way_   -0.152384
┃                   coef_subtour_duration_in_hours_integer_                                                         0.681811
┃                   coef_subtour_return_time_greater_or_equal_to_2pm                                                1.549569
┃                   Length: 188, dtype: float64
┣          loglike: -6094.555918549999
┣        d_loglike: -2.139                                                                                          0.000000
┃                   -3.934                                                                                          0.000000
┃                   coef_alternative_specific_constant_for_outbound_stops_1out_0in                                 -0.000080
┃                   coef_alternative_specific_constant_for_outbound_stops_2out_0in                                 -0.000716
┃                   coef_alternative_specific_constant_for_return_stops_0out_1in                                   -0.000063
┃                                                                                                                     ...   
┃                   coef_primary_destination_accessibility_log_of_it_                                              -0.003978
┃                   coef_subtour_departure_less_than_or_equal_to_11am                                              -0.000790
┃                   coef_subtour_distance_in_miles_from_tour_destination_to_subtour_primary_destination_one_way_   -0.004153
┃                   coef_subtour_duration_in_hours_integer_                                                        -0.002149
┃                   coef_subtour_return_time_greater_or_equal_to_2pm                                                0.000154
┃                   Length: 188, dtype: float64
┣              nit: 130
┣             nfev: 327
┣             njev: 130
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=85, microseconds=611724)
┣           method: 'SLSQP'
┣          n_cases: 5778
┣ iteration_number: 130
┣          logloss: 1.0547864171945307

### Estimated coefficients

In [9]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Like Ratio,Null Value
-2.139,-2.14,0.00,NA,,NA,-2.14
-3.934,-3.93,0.00,NA,,NA,-3.93
coef_alternative_specific_constant_for_outbound_stops_1out_0in,-0.741,0.0909,-8.16,***,NA,0.00
coef_alternative_specific_constant_for_outbound_stops_2out_0in,-2.59,0.167,-15.51,***,NA,0.00
coef_alternative_specific_constant_for_return_stops_0out_1in,-0.447,0.0942,-4.75,***,NA,0.00
coef_alternative_specific_constant_for_return_stops_0out_2in,-1.66,0.136,-12.17,***,NA,0.00
coef_alternative_specific_constant_for_the_total_number_of_stops_0out_2in,0.0394,0.0697,0.57,,NA,0.00
coef_alternative_specific_constant_for_the_total_number_of_stops_2out_3in,0.755,0.335,2.25,*,NA,0.00
coef_am_peak_departure_between_6am_and_7_am_including_interacted_with_outbound_tours,-2.00,0.132,-15.17,***,NA,0.00
coef_dummy_for_all_stops_made_by_transit,-0.673,0.0870,-7.73,***,NA,0.00


# Output Estimation Results

The stop frequency model include seperate coefficient file for each segment,
and has a special writer method to seperate the coefficient by segment
after estimation.

In [10]:
from activitysim.estimation.larch.stop_frequency import update_segment_coefficients
result_dir = data.edb_directory/"estimated"
update_segment_coefficients(
    model, data, result_dir,
    output_file="stop_frequency_coefficients_{segment_name}_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [11]:
for m, segment in zip(model, data.segments):
    m.to_xlsx(
        result_dir/f"{modelname}_{segment}_model_estimation.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `stop_frequency_coefficients_*_revised.csv` files to the configs folder, rename them to `stop_frequency_coefficients_*.csv`, and run ActivitySim in simulation mode.

In [12]:
pd.read_csv(result_dir/"stop_frequency_coefficients_work_revised.csv")

,coefficient_name,Description,value
0,coef_middle_to_low_income_hh,Middle to Low Income HH,0.170000
1,coef_mid_to_high_income_hh,Mid to High Income HH,0.230000
2,coef_high_income_hh,High Income HH,0.240000
3,coef_number_of_hh_persons,Number of HH Persons,-0.328159
4,coef_number_of_students_in_hh,Number of Students in HH,0.315297
5,coef_presence_of_kids_between_0_and_4_includin...,Presence of Kids between 0 and 4 (including) y...,0.557476
6,coef_num_kids_between_5_and_15_including_years...,Num kids between 5 and 15 (including) years old,0.089718
7,coef_presence_of_kids_between_5_and_15_includi...,Presence of kids between 5 and 15 (including) ...,0.117544
8,coef_number_of_adults_16_years_old_,Number of Adults (>= 16 years old),0.074203
9,coef_number_of_cars_number_of_workers,Number of Cars > Number of Workers,-0.065713
